<a href="https://colab.research.google.com/github/cesarmartinezg-lgtm/Entregas/blob/main/04%20-%20enfoque_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importación de Librerías

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Configuración Kaggle

In [ ]:
# Establecer tus credenciales de Kaggle
os.environ['KAGGLE_USERNAME'] = 'cesarmartinezia'
os.environ['KAGGLE_KEY'] = '3bd3c2a5994356c24295cd5c6d8bba59'

In [ ]:
# Crear manualmente el archivo kaggle.json a partir de esas variables
!mkdir -p ~/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (os.environ['KAGGLE_USERNAME'], os.environ['KAGGLE_KEY']))

In [ ]:
# Ajustar permisos
!chmod 600 /root/.kaggle/kaggle.json

# Descarga de Dataset y cargue




In [ ]:
# Descargar los datos de la competencia
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_example.csv  
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
print(f"Train: {df_train.shape}, Test: {df_test.shape}")
test_ids = df_test['ID'].copy()

Train: (692500, 21), Test: (296786, 20)


In [ ]:
# Preprocesamiento
def preprocess(df, is_train=True):
    df = df.copy()
    df = df.drop(columns=[col for col in df.columns if "ID" in col.upper()], errors='ignore')
    y = df.pop('RENDIMIENTO_GLOBAL') if is_train and 'RENDIMIENTO_GLOBAL' in df.columns else None

    for col in df.select_dtypes(include=np.number).columns:
        df[col].fillna(df[col].median(), inplace=True)
    for col in df.select_dtypes(include="object").columns:
        df[col].fillna(df[col].mode()[0] if len(df[col].mode()) > 0 else "DESCONOCIDO", inplace=True)

    return df, y

X_train, y_train = preprocess(df_train, True)
X_test, _ = preprocess(df_test, False)

/tmp/ipython-input-3956138342.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-3956138342.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [ ]:
# FEATURE ENGINEERING SIMPLE (solo 3 features nuevas)
print("\nCreando features...")

def create_features(df):
    df = df.copy()

    # 1. Promedio de indicadores
    indicadores = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
    if all(c in df.columns for c in indicadores):
        df['IND_PROMEDIO'] = df[indicadores].mean(axis=1)

    # 2. Índice tecnológico binario
    if 'F_TIENEINTERNET' in df.columns and 'F_TIENECOMPUTADOR' in df.columns:
        df['TIENE_INTERNET_NUM'] = df['F_TIENEINTERNET'].map({'Si': 1, 'No': 0}).fillna(0)
        df['TIENE_COMP_NUM'] = df['F_TIENECOMPUTADOR'].map({'Si': 1, 'No': 0}).fillna(0)
        df['INDICE_TEC'] = df['TIENE_INTERNET_NUM'] + df['TIENE_COMP_NUM']

    return df

X_train = create_features(X_train)
X_test = create_features(X_test)

print(f"Features: {X_train.shape[1]} (originales + nuevas)")


Creando features...
Features: 22 (originales + nuevas)


In [ ]:
# Encoding (Label Encoding para ser rápido)
for col in X_train.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))

    test_vals = X_test[col].astype(str)
    for val in test_vals.unique():
        if val not in le.classes_:
            le.classes_ = np.append(le.classes_, val)
    X_test[col] = le.transform(test_vals)

# Target encoding
target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
y_encoded = y_train.map(target_map).values

# Escalar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Split
X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_encoded,
                                             test_size=0.2, random_state=42, stratify=y_encoded)

# XGBoost con parámetros equilibrados
print("\nEntrenando XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=100,  # Reducido de 200
    max_depth=5,       # Reducido de 6
    learning_rate=0.1,
    subsample=0.8,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_jobs=-1
)

xgb_model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

print(f"Train Accuracy: {xgb_model.score(X_tr, y_tr):.4f}")
print(f"Val Accuracy: {xgb_model.score(X_val, y_val):.4f}")


Entrenando XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:41:54] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Train Accuracy: 0.4106
Val Accuracy: 0.4039


In [ ]:
# Entrenar con todos los datos
print("\nEntrenamiento final...")
final_xgb = xgb.XGBClassifier(
    n_estimators=100, max_depth=5, learning_rate=0.1,
    subsample=0.8, random_state=42,
    eval_metric='mlogloss', use_label_encoder=False, n_jobs=-1
)
final_xgb.fit(X_train_scaled, y_encoded)

# Predicción
y_pred = final_xgb.predict(X_test_scaled)
inv_map = {v: k for k, v in target_map.items()}
y_pred_labels = [inv_map[p] for p in y_pred]

# Submission
submission = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': y_pred_labels})
submission.to_csv('submission_xgb_04.csv', index=False)

print(f"\n✓ Submission: submission_xgb_04.csv")
print(f"\nDistribución de predicciones:")
print(submission['RENDIMIENTO_GLOBAL'].value_counts())


Entrenamiento final...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:58:00] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



✓ Submission: submission_xgb_04.csv

Distribución de predicciones:
RENDIMIENTO_GLOBAL
alto          98057
bajo          93693
medio-bajo    54070
medio-alto    50966
Name: count, dtype: int64
